<a href="https://colab.research.google.com/github/Inchara67/Dynamic-Pricing-for-Urban-Parking-Lots/blob/main/Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
df = pd.read_csv('/content/dataset.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset.csv'

In [ ]:
# Safely combine 'LastUpdatedDate' and 'LastUpdatedTime' into a datetime column
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'].astype(str) + ' ' + df['LastUpdatedTime'].astype(str),
    format='%d-%m-%Y %H:%M:%S',
    errors='coerce'  # Converts invalid formats to NaT
)

# Drop rows where timestamp couldn't be parsed
df = df.dropna(subset=['Timestamp'])

# Sort the DataFrame by 'Timestamp' and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)


In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity", "TrafficConditionNearby", "QueueLength", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int   # Total parking capacity at the location
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: bool
    VehicleType: str

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
print(df.columns.tolist())

Model 1: Linear Price Model

In [ ]:
# Model 1: Linear pricing based on current occupancy ratio
df['price_model1'] = 10 + 10 * (df['Occupancy'] / df['Capacity'])  # max goes up to $20

base_price = 10
df['price_model1'] = base_price
for i in range(1, len(df)):
    prev_price = df.loc[i - 1, 'price_model1']
    occ = df.loc[i, 'Occupancy']
    cap = df.loc[i, 'Capacity']
    df.loc[i, 'price_model1'] = model1_linear_price(prev_price, occ, cap)

df[['Timestamp','Occupancy', 'Capacity', 'price_model1']].head()

Model 2: Demand-Based Price Function

In [ ]:
def compute_demand(Occupancy, Capacity, QueueLength, TrafficConditionNearby, IsSpecialDay, VehicleType,
                   alpha=0.6, beta=0.2, gamma=0.2, delta=0.1, epsilon=0.1):
    VehicleType = {'car': 1, 'bike': 0.5, 'truck': 1.5}.get(VehicleType.lower(), 1)
    TrafficConditionNearby = {'low': 1, 'medium': 2, 'high': 3}.get(str(TrafficConditionNearby).lower(), 2)


    demand = (alpha * (Occupancy / Capacity) +
              beta * QueueLength -
              gamma * TrafficConditionNearby +
              delta * IsSpecialDay +
              epsilon * VehicleType)
    return demand

def model2_demand_price(base_price, demand, lambda_=0.5):
    norm_demand = min(max(demand, -1), 1)
    price = base_price * (1 + lambda_ * norm_demand)
    return max(min(price, base_price * 2), base_price * 0.5)

df['price_model2'] = base_price
for i in range(len(df)):
    row = df.loc[i]
    demand = compute_demand(
        row['Occupancy'], row['Capacity'], row['QueueLength'], row['TrafficConditionNearby'],
        row['IsSpecialDay'], row['VehicleType']
    )
    df.loc[i, 'price_model2'] = model2_demand_price(base_price, demand)

df[['Timestamp','price_model2']].head()

Model 3: Competitive Pricing Model

In [ ]:
# Safe, bounded linear model
df['price_model1'] = 10 + 10 * (df['Occupancy'] / df['Capacity'])

# Clip prices to 5–20 just in case
df['price_model1'] = df['price_model1'].clip(lower=5, upper=20)


for i in range(len(df)):
    row = df.iloc[i]
    lot = row['SystemCodeNumber']
    competitors = competitor_map.get(lot, [])

    # Find competitors at same time
    comp_prices = df[
        (df['SystemCodeNumber'].isin(competitors)) &
        (df['Timestamp'] == row['Timestamp'])
    ]['price_model2']

    if not comp_prices.empty:
        min_price = comp_prices.min()
        max_price = comp_prices.max()

        if row['Occupancy'] >= row['Capacity'] and row['price_model2'] > min_price:
            df.at[i, 'price_model3'] = max(min_price, 0.5 * row['price_model2'])
        elif row['Occupancy'] < row['Capacity'] and row['price_model2'] < max_price:
            df.at[i, 'price_model3'] = min(max_price + 1, 2 * row['price_model2'])


In [ ]:
df[['Timestamp', 'SystemCodeNumber', 'price_model1', 'price_model2', 'price_model3']].head()
print(df[['Occupancy', 'Capacity']].head(10))
df[['Timestamp', 'SystemCodeNumber', 'Occupancy', 'Capacity', 'price_model1']].head(10)


In [ ]:
import datetime


delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Daily fixed window
        behavior=pw.temporal.exactly_once_behavior()  # Exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # End timestamp of the daily window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy seen in the day
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy seen in the day
        cap=pw.reducers.max(pw.this.Capacity),           # Capacity (assumed constant)
    )
    .with_columns(
        # Model 1: Linear price rise with max occupancy
        price_model1 = 10 + 0.5 * (pw.this.occ_max / pw.this.cap),

        # Original model: price fluctuation based on volatility (peak - low)
        price_fluctuation_model = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --------------------------------------------------------
# Step 1: Ensure SystemCodeNumber is a string
# --------------------------------------------------------
df['SystemCodeNumber'] = df['SystemCodeNumber'].astype(str)

# --------------------------------------------------------
# Step 2: Get all unique parking lot IDs
# --------------------------------------------------------
lot_options = sorted(df['SystemCodeNumber'].unique())

# --------------------------------------------------------
# Step 3: Create a dropdown widget to select a parking lot
# --------------------------------------------------------
dropdown = widgets.Dropdown(options=lot_options, description='Select Lot:')

# --------------------------------------------------------
# Step 4: Define a function to plot prices for the selected lot
# --------------------------------------------------------
def plot_prices(change):

    lot_id = change['new']

    clear_output(wait=True)
    display(dropdown)

    # Filter data for the selected lot
    filtered = df[df['SystemCodeNumber'] == lot_id].sort_values('Timestamp')

    # Create the plot
    plt.figure(figsize=(12, 5))

    # Plot Model 1: original values
    plt.plot(filtered['Timestamp'],
             filtered['price_model1'],
             label='Model 1: Linear',
             color='blue')

    # Plot Model 2: visually nudged down slightly for separation
    plt.plot(filtered['Timestamp'],
             filtered['price_model2'] - 0.3,
             label='Model 2: Demand-Based',
             color='green')

    # Add labels, legend, and formatting
    plt.title(f'Price Trends for Lot {lot_id}')
    plt.xlabel('Timestamp')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plt.show()

# --------------------------------------------------------
# Step 5: Bind the dropdown widget to the plot function
# --------------------------------------------------------
dropdown.observe(plot_prices, names='value')

# --------------------------------------------------------
# Step 6: Display the dropdown and the initial plot
# --------------------------------------------------------
display(dropdown)
plot_prices({'new': lot_options[0]})


In [ ]:
%%capture --no-display
 pw.run()